# KerasTuner

KerasTuner is an easy-to-use, scalable hyperparameter optimization framework that solves the pain points of hyperparameter search. KerasTuner comes with Bayesian Optimization, Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment with new search algorithms.
In this notebook we show how to use the **KerasTuner** for automatic network optimization (and, in general, hyperparameter tuning). This example uses the breast cancer dataset which we have already seen in the course and is completely self contained. However if you want to further understand what's going on please refer to:

* [Official website](https://keras.io/keras_tuner/)
* [Tutorial at Tensorflow](https://www.tensorflow.org/tutorials/keras/keras_tuner)
* [Tutorial at medium.com](https://haneulkim.medium.com/hyperparameter-tuning-with-keras-tuner-full-tutorial-f8128397e857)

# Setup(s)

## Standard libraries setup

In [ ]:
#very common libraries, that we for sure are using
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## KerasTuner setup

In [ ]:
#making sure KerasTuner is installed.
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

## Seed setup

In [ ]:
#resetting the seeds
!wget -O support_code.py https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/lab_day1/support_code.py
%run support_code.py
n1 = 0
reset_random_seeds(n1)

## Data setup

In [ ]:
#libraries for this block
import sklearn.datasets
from sklearn.model_selection import StratifiedShuffleSplit

# loading data
from sklearn.datasets import load_breast_cancer
bcancer = load_breast_cancer()
y = bcancer.target
X = pd.DataFrame(bcancer.data, columns=bcancer.feature_names)

# normalizing
X = (X - X.mean())/X.std()

In [ ]:
#declaring a sss object
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

#sss.split() returns two iterables over the two pieces of data
for train_index, val_index in sss.split(X=X, y=y):
  x_train = X.iloc[train_index, :]
  x_val   = X.iloc[val_index, :]

  y_train = y[train_index]
  y_val   = y[val_index]

# KerasTuner workflow

## Define the (hyper)model

When you build a model for the tuning of the hyperparameters, you also define the **hyperparameter search space** in addition to the model architecture.
The model you set up for fine-tuning is called a **hypermodel**.

With **KerasTuner**, you can define a hypermodel through two approaches:

- by using a **model builder function**
- by subclassing the **HyperModel class** of the KerasTuner API

Here we use a model builder function to define the classification model.
The model builder function returns a compiled model and uses the hyperparameters you define to finetune the model:

- n. of units in the first dense layer
- learning rate

In [ ]:
input_shape = (X.shape[1],)
print(input_shape)

In [ ]:
def model_builder(hp):

  input_shape = (30,)
  model = keras.Sequential()
  model.add(keras.layers.Input(input_shape))
  #model.add(keras.layers.Flatten(input_shape=(30,))) ## n. of features in the BreastCancer dataset

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 16-128
  hp_units = hp.Int('units', min_value=16, max_value=128, step=16)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Hyperparamaters to be explored

The Keras Tuner has four tuners available: i) RandomSearch, ii) Hyperband, iii) BayesianOptimization, iv) Sklearn.
In this tutorial, you use the `Hyperband tuner`.

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (`max_epochs`).

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3, ## factor: Integer, the reduction factor for the number of epochs and number of models for each bracket. Defaults to 3.
                     directory='my_dir',
                     project_name='intro_to_kt')

Create a callback to stop training early after reaching a certain value for the validation loss.



In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

## Train the model

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(x_val, y_val)
print("[test loss, test accuracy]:", eval_result)

# Further steps

The code above is a **very** minimal example and works as a starting point. Stuff to consider:

* each combination of hyperparameter is trained once, with a 70/30 default split. Using `.evaluate_models()` it's possible to do a proper k-fold crossvalidation (see [scan documentation](https://autonomio.github.io/talos/#/Scan), search "evaluate_models")
* the default approach of trying all the combinations can become unfeasible very quickly. The `Scan` function supports several policies for sampling a subset of the hyperparameter space. See the [Towardsdatascience's tutorial](https://towardsdatascience.com/tune-the-hyperparameters-of-your-deep-learning-networks-in-python-using-keras-and-talos-2a2a38c5ac31) for a more in-depth example